In [1]:
from datetime import datetime
import mysql
from mysql.connector import Error
import pandas as pd
import empyrical
import warnings
warnings.filterwarnings("ignore", message="pandas only supports SQLAlchemy connectable")
import pickle
import os


DB_CONFIG = {
    'host': '123.57.74.222',
    'user': 'data_user',
    'password': 'DataUser123',
    'database': 'ai_data'
}

def get_db_connection():
    try:
        connection = mysql.connector.connect(**DB_CONFIG)
        return connection
    except Error as e:
        print(f"Error connecting to MySQL: {e}")
        return None

def get_df_from_sql(query: str):
    connection = get_db_connection()
    if not connection:
        raise Exception("Database connection failed")
    try:
        return pd.read_sql(query, connection)
    except Error as e:
        raise Exception(f"Database error: {e}")
    finally:
        if connection.is_connected():
            connection.close()

df = get_df_from_sql(f"SELECT * FROM fund_info")
fund_code = df['fund_code']
df 
start_date: str = '2020-07-13'
end_date: str = '2025-06-28'
# net_value_df = get_df_from_sql(f"SELECT * FROM net_value WHERE fund_code = '{fund_code}' AND date >= '{start_date}' AND date <= '{end_date}' ORDER BY date asc")
fund_code = fund_code.astype(str)
fund_list = get_df_from_sql(f"SELECT * FROM fund_info")
fund_list

,fund_code,fund_name,qilai_fund_type,qilai_fund_invest_type
0,000001,华夏成长,混合型,平衡
1,000003,中海可转换债券A,债券型,高风险债
2,000004,中海可转换债券C,债券型,高风险债
3,000005,嘉实增强信用,债券型,低风险债
4,000006,西部利得量化成长A,混合型,偏股
...,...,...,...,...
22358,970210,中信建投欣享C,债券型,中短债
22359,970211,中信建投悠享12个月持有A,债券型,低风险债
22360,970212,中信建投悠享12个月持有C,债券型,低风险债
22361,970213,中信建投悦享6个月持有A,债券型,低风险债


In [2]:
start_date: str = '2020-07-13'
end_date: str = '2025-06-28'
fund_code = df['fund_code']
fund_code = fund_code.to_list()
# fund_code = fund_code[:1]
fund_code[:10]

['000001',
 '000003',
 '000004',
 '000005',
 '000006',
 '000008',
 '000009',
 '000010',
 '000011',
 '000013']

In [3]:


# from get_df import faster_get_df_date_as_colum
# def get_fund():
#     all_good_funds = []
#     for i, fund in enumerate(fund_code):
#         print(f"[{i+1}/{len(fund_code)}] 正在处理基金 {fund} ...")

#         df = get_df_from_sql(f"SELECT * FROM net_value WHERE fund_code = '{fund}' AND date >= '{start_date}' AND date <= '{end_date}' ORDER BY date asc")
#         # df = faster_get_df_date_as_colum([fund], "2020-01-01", "2025-07-28")
        
#         if len(df) > 876:
#             print(f"  -> 有效交易日数量: {len(df)}，满足要求，保存中...")
            
#             row = fund_list[fund_list['fund_code'] == fund].iloc[0].tolist()
#             qilai_fund_type = row[-2]
#             qilai_fund_invest_type = row[-1]
            
#             all_good_funds.append(row)

#             folder = f'./fund_data/{qilai_fund_type}_{qilai_fund_invest_type}'
#             os.makedirs(folder, exist_ok=True)
#             file_path = os.path.join(folder, f"{fund}.pkl")
            
#             with open(file_path, 'wb') as f:
#                 pickle.dump(df, f)
#                 print(f"  -> 数据已写入 {file_path}\n")
#         else:
#             print(f"  -> 交易日仅 {len(df)} 天，跳过。\n")
#     # 写入整体结果
#     with open('./fund_data/good_funds.pkl', 'wb') as f:
#         pickle.dump(all_good_funds, f)
#         print("✅ 所有合格基金信息已写入 good_funds.pkl")
#     return all_good_funds
# all_good_funds = get_fund()

In [4]:
def f():
    from tqdm import tqdm

    batch_size = 200
    all_good_funds = []

    for i in range(0, len(fund_code), batch_size):
        batch = fund_code[i:i + batch_size]
        print(f"[{i+1}-{i+len(batch)}/{len(fund_code)}] 正在批量处理基金: {batch}")

        code_str = ','.join([f"'{code}'" for code in batch])
        sql = f"""
            SELECT * FROM net_value 
            WHERE fund_code IN ({code_str}) 
            AND date >= '{start_date}' AND date <= '{end_date}'
            ORDER BY fund_code, date ASC
        """
        df_all = get_df_from_sql(sql)

        # 用 tqdm 显示本批内的进度
        for fund in tqdm(batch, desc=f"处理基金 [{i+1}-{i+len(batch)}]"):
            df = df_all[df_all['fund_code'] == fund]
            
            if len(df) > 876:
                row = fund_list[fund_list['fund_code'] == fund].iloc[0].tolist()
                qilai_fund_type = row[-2]
                qilai_fund_invest_type = row[-1]
                
                all_good_funds.append(row)

                folder = f'./fund_data/{qilai_fund_type}_{qilai_fund_invest_type}'
                os.makedirs(folder, exist_ok=True)
                file_path = os.path.join(folder, f"{fund}.pkl")
                
                with open(file_path, 'wb') as f:
                    pickle.dump(df, f)
            # tqdm 替代 print，无需逐条输出
            

In [5]:
import os
base_dir = './fund_data'
# 获取 base_dir 下所有子文件夹
subfolders = [f for f in os.listdir(base_dir) if os.path.isdir(os.path.join(base_dir, f))]
ans = 0
# 遍历每个子文件夹，统计其中的文件数量
for folder in subfolders:
    folder_path = os.path.join(base_dir, folder)
    files = os.listdir(folder_path)
    ans += len(files)
    print(f"{folder}: {len(files)} 个基金")

债券型_低风险债: 467 个基金
债券型_中短债: 418 个基金
混合型_偏债: 855 个基金
股_场内ETF: 1959 个基金
股票型_主动选股型: 654 个基金
固收_货币: 734 个基金
其他_商品: 1 个基金
QDII型_另类: 25 个基金
债券型_高风险债: 688 个基金
混合型_偏股: 2273 个基金
混合型_平衡: 2000 个基金
FOF型_激进: 47 个基金
FOF型_平衡: 86 个基金
FOF型_求稳: 91 个基金
QDII型_纯债: 53 个基金
QDII型_纯股: 35 个基金
债券型_长债: 1542 个基金
QDII型_混合: 91 个基金


In [ ]:
import os

base_dir = './fund_data'
subfolders = [f for f in os.listdir(base_dir) if os.path.isdir(os.path.join(base_dir, f))]

ans = 0
all_funds = []  # 用于保存 (分组名, 相对路径)

for folder in subfolders:
    folder_path = os.path.join(base_dir, folder)
    files = os.listdir(folder_path)
    count = len(files)
    ans += count
    print(f"{folder}: {count} 个基金")
    
    # 构造路径列表
    for file in files:
        relative_path = os.path.join(folder_path, file)
        all_funds.append((folder, relative_path))
# all_funds

债券型_低风险债: 467 个基金
债券型_中短债: 418 个基金
混合型_偏债: 855 个基金
股_场内ETF: 1959 个基金
股票型_主动选股型: 654 个基金
固收_货币: 734 个基金
其他_商品: 1 个基金
QDII型_另类: 25 个基金
债券型_高风险债: 688 个基金
混合型_偏股: 2273 个基金
混合型_平衡: 2000 个基金
FOF型_激进: 47 个基金
FOF型_平衡: 86 个基金
FOF型_求稳: 91 个基金
QDII型_纯债: 53 个基金
QDII型_纯股: 35 个基金
债券型_长债: 1542 个基金
QDII型_混合: 91 个基金


[('债券型_低风险债', './fund_data/债券型_低风险债/008331.pkl'),
 ('债券型_低风险债', './fund_data/债券型_低风险债/002102.pkl'),
 ('债券型_低风险债', './fund_data/债券型_低风险债/008242.pkl'),
 ('债券型_低风险债', './fund_data/债券型_低风险债/009512.pkl'),
 ('债券型_低风险债', './fund_data/债券型_低风险债/164509.pkl'),
 ('债券型_低风险债', './fund_data/债券型_低风险债/007647.pkl'),
 ('债券型_低风险债', './fund_data/债券型_低风险债/002735.pkl'),
 ('债券型_低风险债', './fund_data/债券型_低风险债/006570.pkl'),
 ('债券型_低风险债', './fund_data/债券型_低风险债/012941.pkl'),
 ('债券型_低风险债', './fund_data/债券型_低风险债/166008.pkl'),
 ('债券型_低风险债', './fund_data/债券型_低风险债/164902.pkl'),
 ('债券型_低风险债', './fund_data/债券型_低风险债/001889.pkl'),
 ('债券型_低风险债', './fund_data/债券型_低风险债/007551.pkl'),
 ('债券型_低风险债', './fund_data/债券型_低风险债/253020.pkl'),
 ('债券型_低风险债', './fund_data/债券型_低风险债/008508.pkl'),
 ('债券型_低风险债', './fund_data/债券型_低风险债/011496.pkl'),
 ('债券型_低风险债', './fund_data/债券型_低风险债/000070.pkl'),
 ('债券型_低风险债', './fund_data/债券型_低风险债/004126.pkl'),
 ('债券型_低风险债', './fund_data/债券型_低风险债/008690.pkl'),
 ('债券型_低风险债', './fund_data/债券型_低风险债/005579.pkl'),


In [7]:
import os

base_dir = './fund_data'
subfolders = [f for f in os.listdir(base_dir) if os.path.isdir(os.path.join(base_dir, f))]

group_file_count = {}
all_funds = []

# 第一次遍历：统计每个分组的文件数量
for folder in subfolders:
    folder_path = os.path.join(base_dir, folder)
    files = os.listdir(folder_path)
    group_file_count[folder] = len(files)

# 第二次遍历：仅收集分组数量 > 100 的文件路径
for folder in subfolders:
    if group_file_count[folder] <= 100:
        continue
    folder_path = os.path.join(base_dir, folder)
    files = os.listdir(folder_path)
    for file in files:
        relative_path = os.path.join(folder_path, file)
        all_funds.append((folder, relative_path))

print(f"共保留 {len(all_funds)} 个文件，来自以下分组：")
for g, c in group_file_count.items():
    if c > 100:
        print(f"{g}: {c} 个")

共保留 11590 个文件，来自以下分组：
债券型_低风险债: 467 个
债券型_中短债: 418 个
混合型_偏债: 855 个
股_场内ETF: 1959 个
股票型_主动选股型: 654 个
固收_货币: 734 个
债券型_高风险债: 688 个
混合型_偏股: 2273 个
混合型_平衡: 2000 个
债券型_长债: 1542 个


In [8]:
import os
import pickle
import numpy as np
from collections import defaultdict
from tslearn.clustering import TimeSeriesKMeans
from tslearn.preprocessing import TimeSeriesScalerMinMax

def dtw_kmeans(X, n_clusters=10, max_iter=10, random_state=42, verbose=True):
    # 先对所有时间序列进行归一化处理
    X_scaled = TimeSeriesScalerMinMax().fit_transform(X)
    cpu_count = os.cpu_count()
    # max_workers = max(1, cpu_count // 4)
    max_workers = 10
    # 使用 tslearn 的 DTW 距离版本的 KMeans
    model = TimeSeriesKMeans(
        n_clusters=n_clusters,
        metric="dtw",
        max_iter=max_iter,
        verbose=verbose,
        random_state=random_state,
        n_jobs=max_workers,
    )
    labels = model.fit_predict(X_scaled)
    return labels

In [ ]:
min_length = 827
base_dir = './fund_data'
save_dir = './fund_data'
os.makedirs(save_dir, exist_ok=True)
max_cluster_size = 70  # 每个簇最多约 100 个样本
min_clusters = 2        # 至少两个簇
max_clusters = 30       # 设置一个最大限制防止过多


# 获取所有子文件夹（分组名）
subfolders = [f for f in os.listdir(base_dir) if os.path.isdir(os.path.join(base_dir, f))]
print("共检测到分组数量：", len(subfolders))

for group in subfolders:
    folder_path = os.path.join(base_dir, group)
    files = os.listdir(folder_path)

    if len(files) <= 100:
        print(f"跳过分组 {group}（仅 {len(files)} 个基金）")
        continue

    print(f"\n开始处理分组: {group}（共 {len(files)} 个文件）")

    X = []
    all_func_code_list = []

    for file in files:
        file_path = os.path.join(folder_path, file)
        try:
            with open(file_path, 'rb') as f:
                now_df = pickle.load(f)
        except Exception as e:
            print(f"读取失败: {file_path}，跳过。原因: {e}")
            continue

        try:
            adj_nav = now_df['adj_nav'].to_numpy().reshape(-1, 1)
        except Exception as e:
            print(f"提取 adj_nav 失败: {file_path}，跳过。原因: {e}")
            continue

        if len(now_df) < min_length:
            continue

        selected_seq = adj_nav[-min_length:, :]
        X.append(selected_seq[:, -1])         # adj_nav 特征序列
        all_func_code_list.append(now_df['fund_code'].to_numpy()[0])  # func_code（此处等于 adj_nav）
        # print(selected_seq.shape, all_func_code_list)
        # break
    # break
    if len(X) == 0:
        print(f"{group}: 无有效数据，跳过。")
        continue

    # 转为 numpy array
    X = np.array(X, dtype=object)
    all_func_code = np.stack(all_func_code_list)
    print(f"已加载 {len(X)} 条时间序列，长度为 {X[0].shape[0]}")
    print(f"{group}: 准备聚类，共 {len(X)} 条序列，序列长度 = {X[0].shape[0]}")

    num_samples = len(X)
    n_clusters = min(max(num_samples // max_cluster_size, min_clusters), max_clusters) + 2

    print(f"{group}: 准备聚类，共 {num_samples} 条序列，设定簇数 = {n_clusters}")

    try:
        labels = dtw_kmeans(X, n_clusters=n_clusters, max_iter=10, verbose=False)
    except Exception as e:
        print(f"聚类失败: {group}，原因: {e}")
        continue

    print("聚类标签:", labels.shape)

    representative_func_codes = all_func_code
    mapping_array = np.column_stack((representative_func_codes, labels))

    save_path = os.path.join(save_dir, f'func_code_to_label_{group}_{n_clusters}.pkl')
    with open(save_path, 'wb') as f:
        pickle.dump(mapping_array, f)

    print(f"映射表已保存至: {save_path}")

共检测到分组数量： 18

开始处理分组: 债券型_低风险债（共 467 个文件）
已加载 467 条时间序列，长度为 827
债券型_低风险债: 准备聚类，共 467 条序列，序列长度 = 827
债券型_低风险债: 准备聚类，共 467 条序列，设定簇数 = 8
聚类标签: (467,)
映射表已保存至: ./fund_data/func_code_to_label_债券型_低风险债_8.pkl

开始处理分组: 债券型_中短债（共 418 个文件）
已加载 418 条时间序列，长度为 827
债券型_中短债: 准备聚类，共 418 条序列，序列长度 = 827
债券型_中短债: 准备聚类，共 418 条序列，设定簇数 = 7
聚类标签: (418,)
映射表已保存至: ./fund_data/func_code_to_label_债券型_中短债_7.pkl

开始处理分组: 混合型_偏债（共 855 个文件）
已加载 855 条时间序列，长度为 827
混合型_偏债: 准备聚类，共 855 条序列，序列长度 = 827
混合型_偏债: 准备聚类，共 855 条序列，设定簇数 = 14
聚类标签: (855,)
映射表已保存至: ./fund_data/func_code_to_label_混合型_偏债_14.pkl

开始处理分组: 股_场内ETF（共 1959 个文件）
已加载 1959 条时间序列，长度为 827
股_场内ETF: 准备聚类，共 1959 条序列，序列长度 = 827
股_场内ETF: 准备聚类，共 1959 条序列，设定簇数 = 29
聚类标签: (1959,)
映射表已保存至: ./fund_data/func_code_to_label_股_场内ETF_29.pkl

开始处理分组: 股票型_主动选股型（共 654 个文件）
已加载 654 条时间序列，长度为 827
股票型_主动选股型: 准备聚类，共 654 条序列，序列长度 = 827
股票型_主动选股型: 准备聚类，共 654 条序列，设定簇数 = 11
聚类标签: (654,)
映射表已保存至: ./fund_data/func_code_to_label_股票型_主动选股型_11.pkl

开始处理分组: 固收_货币（共 734 个文件）
已加载 734 条时间序列，长度为

In [ ]:
# 处理剩下的代码，也生成pkl
import os
import pickle

base_dir = './fund_data'
save_dir = './fund_data'
os.makedirs(save_dir, exist_ok=True)

min_length = 827

# 获取所有子文件夹（分组名）
subfolders = [f for f in os.listdir(base_dir) if os.path.isdir(os.path.join(base_dir, f))]
print("开始处理小分组（基金数 ≤ 100）...")

for group in subfolders:
    folder_path = os.path.join(base_dir, group)
    files = os.listdir(folder_path)

    if len(files) > 100:
        continue  # 跳过大分组，只处理小分组

    fund_codes = []

    for file in files:
        file_path = os.path.join(folder_path, file)
        try:
            with open(file_path, 'rb') as f:
                now_df = pickle.load(f)
        except Exception as e:
            print(f"读取失败: {file_path}，跳过。原因: {e}")
            continue

        if len(now_df) < min_length:
            continue  # 跳过太短的

        try:
            fund_code = now_df['fund_code'].to_numpy()[0]
            fund_codes.append(fund_code)
        except Exception as e:
            print(f"提取 fund_code 失败: {file_path}，跳过。原因: {e}")
            continue

    if not fund_codes:
        print(f"{group}: 无有效 fund_code，跳过")
        continue

    save_path = os.path.join(save_dir, f'func_code_list_{group}.pkl')
    with open(save_path, 'wb') as f:
        pickle.dump(fund_codes, f)

    print(f"{group}: 成功保存 {len(fund_codes)} 个 fund_code 至 {save_path}")

In [ ]:
import os
import pickle
import numpy as np

input_dir = './fund_data'
output_path = './fund_data/func_code_to_label_with_group.pkl'
all_rows = []

label_offset = 0

# 遍历所有 func_code_to_label_<group>_<k>.pkl 文件
for filename in os.listdir(input_dir):
    if not filename.startswith('func_code_to_label_') or not filename.endswith('.pkl'):
        continue

    file_path = os.path.join(input_dir, filename)

    # 提取 group 名称，例如 'func_code_to_label_混合型_偏股_20.pkl' → '混合型_偏股'
    try:
        group_name = filename.replace('func_code_to_label_', '').rsplit('_', 1)[0]
    except Exception:
        print(f"跳过无法解析的文件名: {filename}")
        continue

    with open(file_path, 'rb') as f:
        mapping_array = pickle.load(f)  # shape (N, 2)

    fund_codes = mapping_array[:, 0]
    labels = mapping_array[:, 1].astype(int)

    new_labels = labels + label_offset

    for fund_code, new_label in zip(fund_codes, new_labels):
        all_rows.append([group_name, fund_code, new_label])

    max_label = labels.max()
    label_offset += max_label + 1

    print(f"处理 {filename}: 原始标签 0 ~ {max_label} → 全局标签 {new_labels.min()} ~ {new_labels.max()}")

# 转为 numpy array
final_array = np.array(all_rows, dtype=object)

# 保存
os.makedirs(os.path.dirname(output_path), exist_ok=True)
with open(output_path, 'wb') as f:
    pickle.dump(final_array, f)

print(f"\n✅ 已保存带 group 名的映射至: {output_path}")
print(f"总记录数: {len(final_array)}，全局簇数: {len(np.unique(final_array[:, 2]))}")